# Меня ждут миллионы

Обучите простую модель предсказания цены акций. Используйте один из рассмотренных на лекции "Временные последовательности" алгоритмов предсказания рядов. Можно использовать и более сложные алгоритмы предсказания рядов


1. Найдите данные о цене любого биржевого актива

    Данные должны содержать значение цены актива как минимум за каждый день и длительностю как минимум за год. 

    Найти данные можно на kaggle или использовать FAANG датасет из 1 лабораторной


2. Предобработайте данные

    Временной ряд сложно предсказывать по дням. Сделайте ряд по неделям или месяцам
   
    Удалите из ряда тренд, если он есть. Используйте другие рассмотренные на лекции приёмы


3. Обучите модель
    
    Обучите модель на 80% временного ряда
    
    Предскажите оставшиеся 20% (как минимум 3 точки) ряда с помощью модели 
    
    Отобразите на графике предсказание модели и истинные данные


4. Посчитайте метрику

    Метрика должна отражать разницу между предсказанными и истинными данными. Можно выбрать меру MAPE, можно предложить свою

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.metrics as metrics

from tqdm import tqdm
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.stattools import acf, adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

from sklearn.model_selection import train_test_split

In [ ]:
data_init = pd.read_csv('data/google_stock.csv', parse_dates = [0])

In [ ]:
data_init.info(); print()
print("Shape:", data_init.shape); print()
print(data_init.head(5)); print()
data_init.describe()

In [ ]:
data_init['Date'] = data_init['Date'].dt.strftime('%Y-%U')
data_init

In [ ]:
data = pd.DataFrame(data_init.groupby('Date').High.mean())
data

In [ ]:
plt.figure(figsize=(15, 5))
plt.title('Time series')
plt.xlabel('Date', fontsize=15)
plt.ylabel('Price', rotation=90, fontsize=15)
data['High'].plot();

In [ ]:
fig, ax = plt.subplots(nrows = 4, ncols = 1, figsize = (15, 40))
decompose = seasonal_decompose(data[['High']], period = 13)

ax[0].set_title('Original')
ax[1].set_title('Trend')
ax[2].set_title('Seasonal')
ax[3].set_title('Residual')

decompose.observed.plot(ax = ax[0])
decompose.trend.plot(ax = ax[1])
decompose.seasonal.plot(ax = ax[2])
decompose.resid.plot(ax = ax[3]);

In [ ]:
fig, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (15, 15))
data['High'].plot(ax = ax[0])
plot_acf(data['High'], lags = 24, ax = ax[1]);

In [ ]:
p_value = adfuller(data['High'])[1]
print(p_value)
print("p_value > 0.05? ", p_value > 0.05)

In [ ]:
data['High_log'] = np.log(data['High'])
data.head(5)

In [ ]:
plt.figure(figsize=(15, 10))
plt.ylabel('Logged', rotation=0, labelpad=30)
data['High_log'].plot();

In [ ]:
data['High_diff'] = data['High'] - data['High'].shift(1)
data.head(5)

In [ ]:
plt.subplots(nrows = 3, ncols = 1, figsize = (15, 15))

ax = plt.subplot(311)
ax.set_title('Before')
ax.set_ylabel('High', rotation = 0, labelpad = 30)
data['High'].plot(ax = ax);

ax = plt.subplot(312)
ax.set_title('Logged')
ax.set_ylabel('High_log', rotation = 0, labelpad = 30)
data['High_log'].plot(ax = ax);

ax = plt.subplot(313)
ax.set_title('Diffed')
ax.set_ylabel('High_diff', rotation = 0, labelpad = 30)
data['High_diff'].plot(ax = ax);

In [ ]:
shifts = 1

In [ ]:
data['High_log_diff'] = data['High_log'] - data['High_log'].shift(shifts)
data

In [ ]:
plt.subplots(nrows = 2, ncols = 1, figsize = (15, 15))

ax = plt.subplot(211)
ax.set_title('Before')
ax.set_ylabel('High_log', rotation = 0, labelpad = 30)
data['High_log'].plot(ax = ax);

ax = plt.subplot(212)
ax.set_title('Diffed')
ax.set_ylabel('High_log_diff', rotation = 0, labelpad = 30)
data['High_log_diff'].plot(ax = ax);

In [ ]:
p_log_value = adfuller(data['High_log'])[1]
p_diff_value = adfuller(data['High_diff'][1:])[1]
p_log_diff_value = adfuller(data['High_log_diff'][shifts:])[1]

print("p for log = ", p_log_value)
print("p for diff = ", p_diff_value)
print("p for diffed log = ", p_log_diff_value)

plt.hist(data['High_log_diff'], density = True, bins = 40);

In [ ]:
fig, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (15, 15))
data['High_log_diff'].plot(ax = ax[0])
plot_acf(data['High_log_diff'][shifts:], lags = 26, ax = ax[1]);

In [ ]:
fig, ax = plt.subplots(nrows = 2, ncols = 1, figsize = (15, 20))
decompose = seasonal_decompose(data['High_log_diff'][shifts:], period = 13)

ax[0].set_title('Original')
ax[1].set_title('Trend')

decompose.observed.plot(ax = ax[0])
decompose.trend.plot(ax = ax[1]);

In [ ]:
d=1
D=1

qs = range(1, 4) # 1
Qs = range(1, 3) # 1

ps = range(1, 4) # 1
Ps = range(1, 4) # 1

train_size = 210
test_size = 53

In [ ]:
from itertools import product
parameters = product(ps, qs, Ps, Qs)
parameters_list = list(parameters)
len(parameters_list)

In [ ]:
train = data[:-test_size]

In [ ]:
%%time
import statsmodels.api as sm

results = []
best_aic = float("inf")

for param in tqdm(parameters_list):
    try:
        model = sm.tsa.statespace.SARIMAX(
            train['High_log'], 
            order = (param[0], d, param[1]), 
            seasonal_order = (param[2], D, param[3], 13)
        ).fit(disp = -1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    aic = model.aic
    if aic < best_aic:
        best_model = model
        best_aic = aic
        best_param = param
    results.append([param, model.aic])

In [ ]:
result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'aic']
print(result_table.sort_values(by = 'aic', ascending = True).head())
print(best_model.summary())

In [ ]:
plt.figure(figsize=(15,15))

plt.subplot(211)
best_model.resid[:-test_size].plot()

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(best_model.resid[:-test_size].values.squeeze(), lags=26, ax=ax);

In [ ]:
data['Model'] = np.exp(best_model.fittedvalues)

plt.figure(figsize=(15, 20))

plt.subplot(211)
data['High'][:-test_size].plot()
data['Model'][:-test_size].plot(c = 'r');

plt.subplot(212)
data['High'][13:-test_size].plot()
data['Model'][14:-test_size].plot(c = 'r');

In [ ]:
print(data.shape)
print(data.head(5))
print(data.tail(5))

In [ ]:
plt.figure(figsize=(15,10))
data['High'][13:].plot()
data['Model'][train_size:]= np.exp(best_model.predict(start = train_size, end = train_size + test_size - 1))
   
data['Model'][14:].plot(c = 'y')
data['Model'][14:train_size].plot(c = 'r') 

In [ ]:
def SMAPE(actual, forecasted):
    delim = np.abs(forecasted - actual).sum()
    denom = (forecasted + actual).sum()
    return delim / denom * 100
    
SMAPE(data['High'][train_size:], data['Model'][train_size:])

In [ ]:
print(metrics.mean_absolute_percentage_error(data['High'][train_size:], data['Model'][train_size:]) * 100)